<a href="https://colab.research.google.com/github/Mova-2020/Subworded-Explanatory-Dictionary-on-Physics-/blob/main/Term_comparison_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from IPython import get_ipython
from IPython.display import display

!pip install sentence-transformers
!pip install nltk==3.8.1
!pip install spacy
!python -m spacy download uk_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 55.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('uk_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [28]:
model = SentenceTransformer('all-mpnet-base-v2')
# Load a pre-trained sentence transformer model

Helper functions for each criterion

In [29]:
# 1. Exactness (cosine similarity between term and definition)
def exactness(term, definition):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([term, definition])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return cosine_sim

In [30]:
# 2. Essentiality (ratio of largest and second-largest entailment degrees from definition)
def essentiality(entailments):
    largest = max(entailments)
    second_largest = sorted(entailments, reverse=True)[1]
    return largest / second_largest if second_largest != 0 else float('inf')

In [31]:
# 3. Plainness (ratio of subwords in the term and its definition)
def plainness(term, definition):
    term_subwords = set(term.split())
    definition_subwords = set(definition.split())
    common_subwords = term_subwords.intersection(definition_subwords)
    return len(common_subwords) / len(term_subwords) if term_subwords else 0

In [32]:
# 4. Derivativity (favorable word structure and ending)
def derivativity(term):
    bad_endings = ("nnja", "tja", "ння", "тя")
    num_words = len(term.replace('-', ' ').split())  # Рахуємо слова, розділені пробілами або дефісами

    if term.endswith(bad_endings):
        return 0  # Bad derivation if ending in "nnja", "tja", "ння", "тя"

    if num_words == 1:
        return 1  # Просте слово без закінчень "nnja", "tja", "ння", "тя"
    elif num_words == 2:
        return 0.5  # Два слова без закінчень "nnja", "tja", "ння", "тя"
    else:
        return 0  # Три і більше слова


In [33]:
# 5. Good sound (phonotactic rules compliance)
def good_sound(term):
    bad_clusters = ["ngh", "shr", "zhr", "r r", "stj st", "st st", "нг", "шр", "жр", "р р", "сть ст", "ст ст"]
    if any(cluster in term for cluster in bad_clusters):
        return 0
    # Розбиваємо термін на слова
    words = term.split()

    # Перевіряємо наявність нових кластерів між словами
    for i in range(len(words) - 1):
        if words[i].endswith("р ") and words[i+1].startswith("р"):
            return 0
        if words[i].endswith("сть ") and words[i+1].startswith("ст"):
            return 0
    return 1 # Return 1 if no bad clusters are found

In [34]:
# 6. Systemic feature (existence of hypernyms or hyponyms in the dictionary)
def systemic_feature(term, dictionary_entries):
    related_entries = [entry for entry in dictionary_entries if term in entry]
    return len(related_entries)

In [35]:
# 7. Organicity (inverse of maximum subword length)
def organicity(term):
    subwords = term.split()
    max_length = max(len(subword) for subword in subwords)
    return 1 / max_length if max_length != 0 else 0

In [36]:
# 8. Compatibility (syntactic valence of the term)
def compatibility(term, term_valences):
    return term_valences.get(term, 0)  # Placeholder for valence calculation

In [37]:
# 9. Unambiguity (inverse of number of definitions for the term)
def unambiguity(definitions_count):
    return 1 / definitions_count if definitions_count != 0 else 0

In [38]:
# 10. Nominativity (absence of descriptive structures with prepositions)
def nominativity(term):
    prepositions = ["z ", "zi ", "iz ", "za ", "proty ", "po ", "vid ", "do ", "з ", "зі ", "із ", "за ", "проти ", "по ", "від ", "до "]
    n_conj = sum(1 for prep in prepositions if prep in term)  # Count prepositions in the term
    return 1 / (1 + n_conj)  # Calculate nominativity according to the formula

In [39]:
# 11. Brevity (inverse number of symbols in the term)
def brevity(term):
    return 1 / len(term) if len(term) != 0 else 0

Data input

In [40]:
def get_terms_from_user():
    # Просить користувача ввести терміни та їх означення для порівняння
    while True:
        try:
            num_terms_input = input("Введіть кількість термінів для порівняння (за замовчуванням 2): ")
            num_terms = int(num_terms_input) if num_terms_input else 2  # Use 2 as default if input is empty
            break
        except ValueError:
            print("Неправильний ввід. Будь ласка, введіть число.")

    terms_and_definitions = {}  # Using a dictionary to store terms and definitions
    for i in range(num_terms):
        term = input(f"Введіть термін {i + 1}: ")
        definition = input(f"Введіть означення для терміна '{term}': ")
        terms_and_definitions[term] = {'definition': definition, 'systemic_feature': 0}  # Store term, definition, and initialize systemic_feature

    # Now ask about systemic features after getting all terms and definitions
    for term, data in terms_and_definitions.items():
        while True:
            systemic_input = input(f"Чи наявні в терміносистемі гіпероніми або гіпоніми до терміна '{term}'?\nВведіть '1', якщо такі є, або '0', якщо таких немає: ")
            if systemic_input in ["1", "0"]:
                data['systemic_feature'] = int(systemic_input) # Update systemic_feature in the dictionary
                break
            else:
                print("Неправильний ввід. Будь ласка, введіть 1 або 0.")

    return terms_and_definitions  # Return the dictionary

Scores

In [41]:
def calculate_scores_for_terms(terms_and_definitions):
    # Обчислює бали для кожного терміна

    # Load Ukrainian spaCy model
    nlp = spacy.load("uk_core_news_sm")

    # Assign valence (number of POS connections)
    term_valences = {}
    for term, data in terms_and_definitions.items():
        definition = data['definition'] # Access definition from the nested dictionary
        # Create a sentence with the term and definition for context
        sentence = f"{term} {definition}"
        doc = nlp(sentence)

        # Find the term in the parsed sentence
        term_token = next((token for token in doc if token.text == term), None)

        if term_token:
            # Get POS tags of surrounding words
            surrounding_pos = [token.pos_ for token in term_token.children]

            # Calculate valence as the number of unique POS tags
            valence = len(set(surrounding_pos))
        else:
            valence = 0  # Default valence if term not found

        term_valences[term] = valence

    scores = {}
    for term, data in terms_and_definitions.items():  # Iterate through the nested dictionary
        definition = data["definition"]
        systemic_feature_value = data["systemic_feature"]

        # Calculate Exactness using cosine similarity
        exactness_score = exactness(term, definition)
        print(f"Cosine similarity for '{term}': {exactness_score}") # Print for debugging

        # Essentiality Calculation (placeholder - requires entailment logic)
        # Replace this with your actual entailment calculation
        entailments = [0.8, 0.6, 0.4]  # Example entailment degrees
        essentiality_score = essentiality(entailments)

        # Plainness calculation using NLTK
        term_subwords = set(ngrams(term, len("бозон"))) # Generate subwords of length 4
        definition_subwords = set(ngrams(definition, len("бозон")))
        common_subwords = term_subwords.intersection(definition_subwords) # Find common subwords

        # Convert ngrams back to strings for plainness calculation
        common_subwords_str = {"".join(ngram) for ngram in common_subwords}
        term_subwords_str = {"".join(ngram) for ngram in term_subwords}
        plainness_score = len(common_subwords_str) / len(term_subwords_str) if term_subwords_str else 0

        print(f"Common subwords for '{term}': {common_subwords}")  # Print for debugging

        # Initialize scores for the current term
        scores_term = {
            "Exactness": exactness_score,
            "Essentiality": essentiality_score,
            "Plainness": plainness_score,
            "Derivativity": derivativity(term),
            "Good Sound": good_sound(term),
            # Use the populated dictionary
            "Systemic Feature": systemic_feature_value,  # Access systemic feature value
            "Organicity": organicity(term),
            # Use the populated term_valences
            "Compatibility": compatibility(term, term_valences),
            "Unambiguity": 1,  # Replace with actual calculation based on definition count
            "Nominativity": nominativity(term),  # No prepositionss in the term
            "Brevity": brevity(term),
        }
        scores[term] = scores_term
        #Assign scores_term to scores dictionary with term as key
    return scores


Comparison

In [42]:
1terms_and_definitions = get_terms_from_user() # Get terms and definitions first
scores = calculate_scores_for_terms(terms_and_definitions) # Then calculate scores

Введіть кількість термінів для порівняння (за замовчуванням 2): 4
Введіть термін 1: вода
Введіть означення для терміна 'вода': Прозора безбарвна рідина, що становить собою найпростішу хімічну сполуку водню з киснем. 
Введіть термін 2: камінь
Введіть означення для терміна 'камінь': Прозора безбарвна рідина, що становить собою найпростішу хімічну сполуку водню з киснем. 
Введіть термін 3: радість
Введіть означення для терміна 'радість': Прозора безбарвна рідина, що становить собою найпростішу хімічну сполуку водню з киснем. 
Введіть термін 4: бігати
Введіть означення для терміна 'бігати': Прозора безбарвна рідина, що становить собою найпростішу хімічну сполуку водню з киснем. 
Чи наявні в терміносистемі гіпероніми або гіпоніми до терміна 'вода'?
Введіть '1', якщо такі є, або '0', якщо таких немає: 1
Чи наявні в терміносистемі гіпероніми або гіпоніми до терміна 'камінь'?
Введіть '1', якщо такі є, або '0', якщо таких немає: 
Неправильний ввід. Будь ласка, введіть 1 або 0.
Чи наявні в термі

In [43]:
for term, term_scores in scores.items():
    print(f"Бали для терміна '{term}': {term_scores}")
    overall_score = sum(term_scores.values()) / len(term_scores)
    print(f"Загальний бал для терміна '{term}': {overall_score}")

Бали для терміна 'вода': {'Exactness': 0.0, 'Essentiality': 1.3333333333333335, 'Plainness': 0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 1, 'Organicity': 0.25, 'Compatibility': 1, 'Unambiguity': 1, 'Nominativity': 1.0, 'Brevity': 0.25}
Загальний бал для терміна 'вода': 0.7121212121212122
Бали для терміна 'камінь': {'Exactness': 0.0, 'Essentiality': 1.3333333333333335, 'Plainness': 0.0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 1, 'Organicity': 0.16666666666666666, 'Compatibility': 1, 'Unambiguity': 1, 'Nominativity': 1.0, 'Brevity': 0.16666666666666666}
Загальний бал для терміна 'камінь': 0.6969696969696971
Бали для терміна 'радість': {'Exactness': 0.0, 'Essentiality': 1.3333333333333335, 'Plainness': 0.0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 1, 'Organicity': 0.14285714285714285, 'Compatibility': 3, 'Unambiguity': 1, 'Nominativity': 1.0, 'Brevity': 0.14285714285714285}
Загальний бал для терміна 'радість': 0.8744588744588746
Бали для термі